In [6]:
import numpy as np
import re
import copy
from io import TextIOWrapper
import matplotlib.pyplot as plt

ang2au = 1.8897259886

In [7]:
file = "ORMAS_SYM/L4_Small/mcscf.dat"

fname = open(file, 'r')

if isinstance(fname, TextIOWrapper):
    flines = fname.readlines()

In [8]:
SYM = " ".join(flines[2].split())
SYM

'CNV 2'

In [9]:
VEC_SWITCH = 0

MOs = []
AOs = ""

VECs = dict()
VECi = -1

for i,line in enumerate(flines):
    thisline = line.split()
    
    if len(thisline) == 0:
        continue
        
    if VEC_SWITCH:
        if line[:2] != MO_IND:
            MO_IND = line[:2]
            AOs = AOs.strip()
            if AOs[0] != "-":
                AOs = " "+AOs
            AOs = re.findall('.'*15,AOs)
            AOs = [float(x) for x in AOs]
            MOs.append(AOs)
            AOs = ""
        AOs += line[5:-1]
                
    if thisline[0] == "$VEC":
        VEC_SWITCH = 1
        MO_IND = " 1"
        continue
    if thisline[0] == "$END":
        VEC_SWITCH = 0
        VECs[VECi] = MOs
        VECi += 1
        MOs = []
        continue
        
del VECs[-1]

In [11]:
VEC = np.array(VECs[1])

In [12]:
len(VEC[0])

641

In [13]:
# Auto Find Sym appraoch, or +- by hand

In [14]:
# count number of S, P, D
# Make px py pz, etc
# account for sym (Double all but Fe)

 ## Could do prior automatically, give coords, do ops 
 ## [only need to do one, C2, or apply both reflections, both are easy]
 ## if coord is unique, add, then repeat last set of AOs
 ## This would give what sym uniques are connected

# Sym Ops will transpose and sign flip, set transpositions by hand

# Only need sym unique MOs, rest can be generated by applying C2, check if unique, 
# if NOT, take sum and difference, normalize, make those the original and sym copy

In [15]:
BASIS_INFO = []
ATOM_INFO = []

for i,line in enumerate(flines):
    thisline = line.split()
    if len(thisline) == 0:
        continue
    if thisline[0] == "$END":
        BASIS_INFO.append(ATOM_INFO)
        break
    if len(thisline) == 5:
        BASIS_INFO.append(ATOM_INFO)
        ATOM_INFO = []
        ATOM_INFO.append(thisline)
        ATOM_INFO[0][2] = float(ATOM_INFO[0][2])
        ATOM_INFO[0][3] = float(ATOM_INFO[0][3])
        ATOM_INFO[0][4] = float(ATOM_INFO[0][4])
        ATOM_INFO.append([])
    if thisline[0] == "S":
        ATOM_INFO[1].extend(["S"])
    if thisline[0] == "P":
        ATOM_INFO[1].extend(["Px", "Py", "Pz"])
    if thisline[0] == "D":
        ATOM_INFO[1].extend(["Dxx", "Dyy", "Dzz", "Dxy", "Dxz", "Dyz"])
    if thisline[0] == "F":
        ATOM_INFO[1].extend(["Fxxx", "Fyyy", "Fzzz", "Fxxy", "Fxxz", "Fxyy", "Fyyz", "Fxzz", "Fyzz", "Fxyz"])

del BASIS_INFO[0]

NEW_BASIS_INFO = []
SYM_GROUPS = []
C2_reorder = []

i = 0
for C in BASIS_INFO:
    NEW_BASIS_INFO.append(C)
    
    RC = copy.deepcopy(C)
    RC[0][2] = -1*C[0][2]
    RC[0][3] = -1*C[0][3]
#     print(NEW_BASIS_INFO[-1])
    if RC not in NEW_BASIS_INFO:
        SYM_GROUPS.append([i,i+1])
        NEW_BASIS_INFO.append(RC)
        C2_reorder.extend([i+1,i])
        i += 2
    else:
        C2_reorder.append(i)
        i += 1    
    
BASIS_INFO = NEW_BASIS_INFO  

print(SYM_GROUPS)
print(C2_reorder)

[[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11, 12], [13, 14], [15, 16], [17, 18], [19, 20], [21, 22], [23, 24], [25, 26], [27, 28]]
[0, 2, 1, 4, 3, 6, 5, 8, 7, 10, 9, 12, 11, 14, 13, 16, 15, 18, 17, 20, 19, 22, 21, 24, 23, 26, 25, 28, 27]


In [29]:
print(BASIS_INFO[19])

[['N', '7.0', 3.12870257, 0.0, 1.0527659], ['S', 'S', 'S', 'S', 'Px', 'Py', 'Pz', 'Px', 'Py', 'Pz', 'Px', 'Py', 'Pz', 'Dxx', 'Dyy', 'Dzz', 'Dxy', 'Dxz', 'Dyz', 'Dxx', 'Dyy', 'Dzz', 'Dxy', 'Dxz', 'Dyz']]


In [17]:
C2_coeffs = []

for fs in BASIS_INFO:
    cs = []
    for f in fs[1]:
        c = 1
        for ch in f:
            if ch == "x":
                c *= -1 #???
            if ch == "y":
                c *= -1
            if ch == "z":
                c *= 1
        cs.append(c)
    C2_coeffs.append(np.array(cs))
    
C2_coeffs[0]

array([ 1,  1,  1,  1,  1,  1,  1, -1, -1,  1, -1, -1,  1, -1, -1,  1, -1,
       -1,  1, -1, -1,  1, -1, -1,  1,  1,  1,  1,  1, -1, -1,  1,  1,  1,
        1, -1, -1,  1,  1,  1,  1, -1, -1,  1,  1,  1,  1, -1, -1, -1, -1,
        1, -1,  1, -1,  1, -1, -1,  1, -1, -1,  1, -1,  1, -1,  1, -1, -1,
        1])

In [460]:
ATOM_INDEX = [0]
BASIS_FUNCS = []

for C in BASIS_INFO:
    ATOM_INDEX.append(ATOM_INDEX[-1]+len(C[1]))
    BASIS_FUNCS.extend(C[1])
    
#ATOM_INDEX.pop()
BASIS_FUNCS

['S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'Px',
 'Py',
 'Pz',
 'Px',
 'Py',
 'Pz',
 'Px',
 'Py',
 'Pz',
 'Px',
 'Py',
 'Pz',
 'Px',
 'Py',
 'Pz',
 'Px',
 'Py',
 'Pz',
 'Dxx',
 'Dyy',
 'Dzz',
 'Dxy',
 'Dxz',
 'Dyz',
 'Dxx',
 'Dyy',
 'Dzz',
 'Dxy',
 'Dxz',
 'Dyz',
 'Dxx',
 'Dyy',
 'Dzz',
 'Dxy',
 'Dxz',
 'Dyz',
 'Dxx',
 'Dyy',
 'Dzz',
 'Dxy',
 'Dxz',
 'Dyz',
 'Fxxx',
 'Fyyy',
 'Fzzz',
 'Fxxy',
 'Fxxz',
 'Fxyy',
 'Fyyz',
 'Fxzz',
 'Fyzz',
 'Fxyz',
 'Fxxx',
 'Fyyy',
 'Fzzz',
 'Fxxy',
 'Fxxz',
 'Fxyy',
 'Fyyz',
 'Fxzz',
 'Fyzz',
 'Fxyz',
 'S',
 'S',
 'S',
 'S',
 'Px',
 'Py',
 'Pz',
 'Px',
 'Py',
 'Pz',
 'Px',
 'Py',
 'Pz',
 'Dxx',
 'Dyy',
 'Dzz',
 'Dxy',
 'Dxz',
 'Dyz',
 'Dxx',
 'Dyy',
 'Dzz',
 'Dxy',
 'Dxz',
 'Dyz',
 'S',
 'S',
 'S',
 'S',
 'Px',
 'Py',
 'Pz',
 'Px',
 'Py',
 'Pz',
 'Px',
 'Py',
 'Pz',
 'Dxx',
 'Dyy',
 'Dzz',
 'Dxy',
 'Dxz',
 'Dyz',
 'Dxx',
 'Dyy',
 'Dzz',
 'Dxy',
 'Dxz',
 'Dyz',
 'S',
 'S',
 'S',
 'S',
 'Px',
 'Py',
 'Pz',
 'Px',
 'Py',
 'Pz',
 'Px',
 'Py',
 'Pz',
 'Dxx',

In [461]:
aVECs = []
aVEC = []

for v in VEC:
    for i in range(len(ATOM_INDEX)-1):
        aVEC.append(np.array(v[ATOM_INDEX[i]:ATOM_INDEX[i+1]]))
    aVECs.append(aVEC)
    aVEC=[]

In [462]:
print(SYM_GROUPS)
print(aVECs[0][1])
print(BASIS_INFO[1][1])

[[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11, 12], [13, 14], [15, 16], [17, 18], [19, 20], [21, 22], [23, 24], [25, 26], [27, 28]]
[-2.26353725e-04 -1.82920461e-03 -1.60884081e-03  8.96365402e-03
  0.00000000e+00 -1.03750943e-03 -1.18877374e-03  0.00000000e+00
  1.40749573e-03  1.57195779e-03  0.00000000e+00  2.11916701e-04
  2.58771499e-03  1.30486086e-04 -2.32876457e-05 -1.07198440e-04
  0.00000000e+00  0.00000000e+00 -2.51603054e-04 -2.77179882e-04
 -5.02446558e-04  7.79626440e-04  0.00000000e+00  0.00000000e+00
  1.28108184e-03]
['S', 'S', 'S', 'S', 'Px', 'Py', 'Pz', 'Px', 'Py', 'Pz', 'Px', 'Py', 'Pz', 'Dxx', 'Dyy', 'Dzz', 'Dxy', 'Dxz', 'Dyz', 'Dxx', 'Dyy', 'Dzz', 'Dxy', 'Dxz', 'Dyz']


In [463]:
print(aVECs[0][2])
print(BASIS_INFO[2][1])

[-2.26353725e-04 -1.82920461e-03 -1.60884081e-03  8.96365402e-03
  0.00000000e+00  1.03750943e-03 -1.18877374e-03  0.00000000e+00
 -1.40749573e-03  1.57195779e-03  0.00000000e+00 -2.11916701e-04
  2.58771499e-03  1.30486086e-04 -2.32876457e-05 -1.07198440e-04
  0.00000000e+00  0.00000000e+00  2.51603054e-04 -2.77179882e-04
 -5.02446558e-04  7.79626440e-04  0.00000000e+00  0.00000000e+00
 -1.28108184e-03]
['S', 'S', 'S', 'S', 'Px', 'Py', 'Pz', 'Px', 'Py', 'Pz', 'Px', 'Py', 'Pz', 'Dxx', 'Dyy', 'Dzz', 'Dxy', 'Dxz', 'Dyz', 'Dxx', 'Dyy', 'Dzz', 'Dxy', 'Dxz', 'Dyz']


In [464]:
print(SYM_GROUPS)
print(aVECs[124][1])
print(BASIS_INFO[1][1])

[[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11, 12], [13, 14], [15, 16], [17, 18], [19, 20], [21, 22], [23, 24], [25, 26], [27, 28]]
[ 6.82672771e-10  2.62836963e-09 -1.47365225e-08  6.14855366e-09
 -5.06805087e-02  9.16256443e-12  1.22106129e-10  1.07077216e-02
  9.61477297e-09  9.37853444e-09 -4.12829798e-02  1.59308481e-08
 -6.28864257e-09 -2.37809338e-10  2.05226826e-10  3.25825108e-11
  1.12993876e+00  8.83027348e-02  5.60964635e-10  1.07609482e-09
 -1.14951494e-09  7.34201098e-11 -3.32981110e-02  6.51001324e-03
  2.03612621e-10]
['S', 'S', 'S', 'S', 'Px', 'Py', 'Pz', 'Px', 'Py', 'Pz', 'Px', 'Py', 'Pz', 'Dxx', 'Dyy', 'Dzz', 'Dxy', 'Dxz', 'Dyz', 'Dxx', 'Dyy', 'Dzz', 'Dxy', 'Dxz', 'Dyz']


In [465]:
print(aVECs[125][2])
print(BASIS_INFO[2][1])

[-6.80401203e-10 -2.54309115e-09  1.78285101e-08 -2.86766489e-08
  5.06805188e-02  1.03081264e-10 -7.59728032e-11 -1.07076614e-02
  7.35403113e-09 -7.80858204e-09  4.12830001e-02  1.19829553e-09
 -5.26519210e-09  1.95964037e-10 -1.71317197e-10 -2.46468359e-11
  1.12993866e+00 -8.83035854e-02  5.34809782e-10 -6.54258731e-10
  1.96925171e-09 -1.31499298e-09 -3.32981341e-02 -6.50998570e-03
  1.54857972e-09]
['S', 'S', 'S', 'S', 'Px', 'Py', 'Pz', 'Px', 'Py', 'Pz', 'Px', 'Py', 'Pz', 'Dxx', 'Dyy', 'Dzz', 'Dxy', 'Dxz', 'Dyz', 'Dxx', 'Dyy', 'Dzz', 'Dxy', 'Dxz', 'Dyz']


In [466]:
V = aVECs[124]

In [467]:
v = copy.deepcopy(aVECs[124])
new_v = []

for i,atom_vecs in enumerate(v):
    atom_vecs *= C2_coeffs[i]
for i in C2_reorder:
    new_v.append(v[i])

new_v[2]
  

array([ 6.82672771e-10,  2.62836963e-09, -1.47365225e-08,  6.14855366e-09,
        5.06805087e-02, -9.16256443e-12,  1.22106129e-10, -1.07077216e-02,
       -9.61477297e-09,  9.37853444e-09,  4.12829798e-02, -1.59308481e-08,
       -6.28864257e-09, -2.37809338e-10,  2.05226826e-10,  3.25825108e-11,
        1.12993876e+00, -8.83027348e-02, -5.60964635e-10,  1.07609482e-09,
       -1.14951494e-09,  7.34201098e-11, -3.32981110e-02, -6.51001324e-03,
       -2.03612621e-10])

In [468]:
p_v[2]

array([-6.80401203e-10, -2.54309115e-09,  1.78285101e-08, -2.86766489e-08,
        5.06805188e-02,  1.03081264e-10, -7.59728032e-11, -1.07076614e-02,
        7.35403113e-09, -7.80858204e-09,  4.12830001e-02,  1.19829553e-09,
       -5.26519210e-09,  1.95964037e-10, -1.71317197e-10, -2.46468359e-11,
        1.12993866e+00, -8.83035854e-02,  5.34809782e-10, -6.54258731e-10,
        1.96925171e-09, -1.31499298e-09, -3.32981341e-02, -6.50998570e-03,
        1.54857972e-09])

In [469]:
cu = 0

p_v = copy.deepcopy(aVECs[125])

for i in range(len(p_v)):
    cu += sum(p_v[i]*new_v[i])
    print(cu)
    
cu

0.011756774855608963
0.012062941728086057
1.3021604776169
1.3066877178413114
1.3252698092733128
1.3253330603006905
1.438469871499841
1.43849279856175
1.4489029340909103
1.4490942617969897
1.4564904623256842
1.4565890394668093
1.4582130785023677
1.458679979530158
1.4588805787920596
1.459026431386703
1.4590391805515623
1.466496517391842
1.473953854533807
1.4772877513367566
1.4806216482774368
1.4806225203167724
1.48091590994074
1.480917088083586
1.4810510289365642
1.4810586488487725
1.4810775598565098
1.4810776224858824
1.4810785577612062


1.4810785577612062

In [470]:
rVECs = []

for i in range(len(aVECs)):
    v = copy.deepcopy(aVECs[i])
    new_v = []
    for j,atom_vecs in enumerate(v):
        atom_vecs *= C2_coeffs[j]
    for j in C2_reorder:
        new_v.append(v[j])
    rVECs.append(new_v)

    

In [423]:
aVECs[127][3]

array([-1.46187010e-01, -6.13761794e-01,  1.36694743e+00, -1.18467723e+00,
       -8.77752672e-10, -3.85918075e-01,  2.13238521e-02, -1.25545395e-09,
        3.37032144e+00,  9.09383677e-01, -1.52756674e-08, -1.16491279e+00,
       -1.39471494e+00,  7.31690711e-02, -8.88105525e-02,  1.56414814e-02,
       -9.02946761e-10, -6.32931464e-10, -3.39126230e-02, -7.21727395e-02,
        9.83119433e-01, -9.10946693e-01,  7.74496327e-09, -4.70793860e-09,
        4.54627704e-01])

In [424]:
aVECs[127][4]

array([ 4.08495661e-01,  1.78610800e+00, -1.27971133e+01,  3.28286067e+00,
       -3.46874893e-09,  3.19076222e-03, -2.63393488e-02,  2.74186213e-10,
       -6.59673189e-01,  2.82104720e+00, -1.65349798e-08,  9.37737510e-02,
       -1.10321415e+00, -1.07480055e+00,  5.34422845e-01,  5.40377706e-01,
       -3.69153182e-10, -3.58635032e-09, -1.17283413e-02,  7.91393834e-02,
       -2.10563622e-01,  1.31424238e-01,  5.18399557e-09,  4.38383999e-09,
        1.50926487e+00])

In [425]:
rVECs[126][3]

array([ 1.47038085e-01,  6.16719916e-01, -1.40518270e+00,  7.74452459e-01,
       -8.93544832e-10,  3.86289358e-01, -2.15987854e-02, -2.75897784e-09,
       -3.27798087e+00, -8.74933148e-01, -8.49090232e-09,  1.32243228e+00,
        1.46984027e+00, -7.32016098e-02,  8.94078234e-02, -1.62062136e-02,
       -9.73542321e-10, -9.10534436e-10,  3.39123746e-02,  7.58903525e-02,
       -9.72701949e-01,  8.96811597e-01,  3.31313046e-09, -4.12643179e-09,
       -4.80853452e-01])

In [426]:
rVECs[126][4]

array([-4.07052180e-01, -1.78116611e+00,  1.27114457e+01, -3.81246435e+00,
       -3.77130041e-09, -3.84900007e-03,  2.59431410e-02,  6.10531769e-09,
        7.02433212e-01, -2.81728165e+00, -3.98919825e-09, -2.59456560e-01,
        1.03933514e+00,  1.07501300e+00, -5.34237415e-01, -5.40775583e-01,
       -2.77493633e-10, -3.80906261e-09,  1.16694291e-02, -7.55847280e-02,
        1.89002184e-01, -1.13417456e-01,  2.66299845e-09,  6.12353150e-09,
       -1.52726233e+00])

In [427]:
cu_grid = np.zeros((len(aVECs),len(aVECs)))
MO_PAIRS = []

for i in range(len(aVECs)):
    for j in range(i):
        cu1 = 0
        cu2 = 0
        for atom in range(len(aVECs[0])):
            cu1 += sum((aVECs[i][atom] - rVECs[j][atom])**2)
        for atom in range(len(aVECs[0])):
            cu2 += sum((aVECs[i][atom] + rVECs[j][atom])**2)
        cu_grid[i,j] += abs(min(cu1,cu2))
        if cu_grid[i,j] < 3:
            MO_PAIRS.append([j,i])
print(MO_PAIRS)

[[0, 1], [0, 2], [1, 2], [0, 3], [1, 3], [2, 3], [0, 4], [1, 4], [2, 4], [3, 4], [0, 5], [1, 5], [2, 5], [3, 5], [4, 5], [0, 6], [1, 6], [2, 6], [3, 6], [4, 6], [5, 6], [0, 7], [1, 7], [2, 7], [3, 7], [4, 7], [5, 7], [6, 7], [0, 8], [1, 8], [2, 8], [3, 8], [4, 8], [5, 8], [6, 8], [7, 8], [0, 9], [1, 9], [2, 9], [3, 9], [4, 9], [5, 9], [6, 9], [7, 9], [8, 9], [0, 10], [1, 10], [2, 10], [3, 10], [4, 10], [5, 10], [6, 10], [7, 10], [8, 10], [9, 10], [0, 11], [1, 11], [2, 11], [3, 11], [4, 11], [5, 11], [6, 11], [7, 11], [8, 11], [9, 11], [10, 11], [0, 12], [1, 12], [2, 12], [3, 12], [4, 12], [5, 12], [6, 12], [7, 12], [8, 12], [9, 12], [10, 12], [11, 12], [0, 13], [1, 13], [2, 13], [3, 13], [4, 13], [5, 13], [6, 13], [7, 13], [8, 13], [9, 13], [10, 13], [11, 13], [12, 13], [0, 14], [1, 14], [2, 14], [3, 14], [4, 14], [5, 14], [6, 14], [7, 14], [8, 14], [9, 14], [10, 14], [11, 14], [12, 14], [13, 14], [0, 15], [1, 15], [2, 15], [3, 15], [4, 15], [5, 15], [6, 15], [7, 15], [8, 15], [9, 15],

In [471]:
SYM_MOS = []

for i in range(len(aVECs)):
    cu1 = 0
    cu2 = 0
    for atom in range(len(aVECs[0])):
        cu1 += sum((aVECs[i][atom] - rVECs[i][atom])**2)
    for atom in range(len(aVECs[0])):
        cu2 += sum((aVECs[i][atom] + rVECs[i][atom])**2)
    cu = abs(min(cu1,cu2))
    print(i+1,cu1, cu2)
    if cu < 1:
        SYM_MOS.append(i+1)
print(SYM_MOS)

1 0.0 4.089571170978952
2 0.0 4.022597543894208
3 0.0 4.0332800960042015
4 4.030544472391165 0.0
5 4.035834051963472 0.0
6 4.059144109432063 0.0
7 0.0 4.077806683973976
8 0.0 4.038469113390116
9 4.0295843744122255 0.0
10 4.0419554128919675 0.0
11 0.0 4.04029831289632
12 0.0 4.124591510571975
13 4.0874252668423905 0.0
14 4.211826382957847 0.0
15 0.0 4.087925819881806
16 4.100647366311429 0.0
17 0.0 4.077325174726239
18 4.2288972395994175 0.0
19 0.0 4.109159611839851
20 4.125411820160844 0.0
21 0.0 4.1223643896841615
22 4.039466245525224 0.0
23 0.0 4.039987722441593
24 4.053985358405283 0.0
25 0.0 4.054659112287625
26 0.0 4.298858540322314
27 0.0 8.281335969777077
28 8.301712938664464 0.0
29 7.094295607982195 0.0
30 0.0 6.96017933569901
31 7.575208191173367 0.0
32 0.0 11.479093934368752
33 11.703079420652069 0.0
34 0.0 12.365814599873787
35 0.0 15.745946856883252
36 6.987120308200436 0.0
37 0.0 12.256607939503594
38 14.277560798237532 0.0
39 8.23809349369779 0.0
40 0.0 8.320160402847467


In [472]:
aVECs[586][0]

array([-2.77760058e-01, -8.36704268e-01, -4.21017307e+00, -1.31335594e+01,
        9.14003302e-01,  1.48593846e+02,  9.70525863e+00,  4.56346265e-09,
        1.63766177e-03, -1.58103000e-01,  7.07333698e-08,  2.84704113e-02,
       -2.67249353e+00,  1.19838324e-06, -2.42907655e-01,  1.90370077e+01,
       -1.47153267e-06,  4.74685827e-01, -3.87846133e+01,  1.25112329e-05,
       -4.29538503e-01,  1.36321316e+01,  5.74699398e-08, -2.35139819e-02,
       -9.48223642e-01,  1.16596053e-01,  1.63293041e-03, -1.18228983e-01,
        9.43623314e-11,  6.13199161e-09, -3.29426071e-04,  1.55118434e-01,
        6.69370229e-03, -1.61812137e-01,  1.01201123e-10,  8.91485169e-09,
       -4.48250406e-04,  1.09958539e+00, -7.91791662e-01, -3.07793730e-01,
        8.63113450e-09, -7.03993724e-09,  1.96583533e-01, -5.78124167e+01,
        3.02381795e+01,  2.75742372e+01,  1.09016531e-07, -1.57520932e-06,
       -1.04255227e-01,  4.74890229e-09, -2.06467251e-05, -7.08011777e-03,
        3.22869004e-04, -

In [474]:
rVECs[586][0]

array([-2.77760058e-01, -8.36704268e-01, -4.21017307e+00, -1.31335594e+01,
        9.14003302e-01,  1.48593846e+02,  9.70525863e+00, -4.56346265e-09,
       -1.63766177e-03, -1.58103000e-01, -7.07333698e-08, -2.84704113e-02,
       -2.67249353e+00, -1.19838324e-06,  2.42907655e-01,  1.90370077e+01,
        1.47153267e-06, -4.74685827e-01, -3.87846133e+01, -1.25112329e-05,
        4.29538503e-01,  1.36321316e+01, -5.74699398e-08,  2.35139819e-02,
       -9.48223642e-01,  1.16596053e-01,  1.63293041e-03, -1.18228983e-01,
        9.43623314e-11, -6.13199161e-09,  3.29426071e-04,  1.55118434e-01,
        6.69370229e-03, -1.61812137e-01,  1.01201123e-10, -8.91485169e-09,
        4.48250406e-04,  1.09958539e+00, -7.91791662e-01, -3.07793730e-01,
        8.63113450e-09,  7.03993724e-09, -1.96583533e-01, -5.78124167e+01,
        3.02381795e+01,  2.75742372e+01,  1.09016531e-07,  1.57520932e-06,
        1.04255227e-01, -4.74890229e-09,  2.06467251e-05, -7.08011777e-03,
       -3.22869004e-04, -

array([ 3.86219200e-04,  1.15655670e-03,  5.32397600e-03,  1.27195660e-02,
        8.48487490e-04,  1.34796127e-01, -8.44085710e-03, -2.40361796e-11,
        2.74080500e-05,  2.64605730e-04, -3.92579830e-10,  4.91052400e-04,
        4.53602490e-03,  1.10710600e-09, -5.49773200e-03, -2.95451200e-02,
       -3.10493300e-09,  1.01449260e-02,  6.19687100e-02,  7.63768538e-08,
       -2.63092600e-02, -4.31096050e-02,  3.82466280e-10, -1.36274821e-03,
        3.87347110e-03, -1.24115700e-04,  2.05369800e-05,  1.03578700e-04,
        1.76988410e-11,  6.07677100e-11,  5.12594500e-05, -1.49796300e-04,
        2.46397500e-05,  1.25156600e-04,  3.55699060e-11,  6.53199000e-11,
        6.19336500e-05,  1.95100642e-02,  1.81888200e-03, -2.13289470e-02,
       -9.27797070e-10, -4.23777312e-09,  5.81707090e-03, -5.36080022e-02,
        1.49119559e-02,  3.86960460e-02,  8.65765824e-09, -1.02665402e-08,
       -7.68869700e-03, -8.07353600e-12,  1.98338400e-05, -2.54035700e-05,
       -1.13675200e-05,  

In [347]:
already_sym = []

for i in range(588):
    if not any(i in x for x in MO_PAIRS):
        already_sym.append(i+1)
already_sym

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 203,
 254,
 255,
 256,
 257,
 258,
 267,
 294,
 303,
 304,
 305,
 328,
 339,
 344,
 345,
 350,
 351,
 394,
 395,
 432,
 433,
 438,
 439,
 442,
 443,
 452,
 453,
 464,
 465,
 468,
 469,
 472,
 473,
 476,
 477,
 478,
 481,
 482,
 483,
 484,
 485,
 486,
 487,
 488,
 489,
 490,
 493,
 494,
 495,
 496,
 497,
 498,
 501,
 502,
 503,
 504,
 505,
 506,
 507,
 508,
 509

In [348]:
cu_grid[304,303]

2.6706651951702782

In [217]:
### ROTATE VECS (Combine into lists of 2, function and value?)
### Get Vecs AND rotated VECS, then a basic overlap check